# Classification and Model Selection Group 6 - Donatus

### Kickstarter Campaign Prediction

Import Dataset

In [92]:
import os
import numpy as np
os.getcwd() 

'/Users/donatusschaumburg/Documents/Data Science Education/Cambridge Spark/Course Material/ADS03'

In [93]:
import pandas as pd
from pandas_profiling import ProfileReport

#prepare the Data and combine columns
df = pd.read_csv("data/kickstarter.csv")
df["goal_usd"] = df["goal"] * df["static_usd_rate"]
df["campaign_length"] = df["deadline"] - df["launched_at"]
df['category'] = df['category'].str.extract(r'("slug":")(.*)(/)')[1]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      60000 non-null  int64  
 1   photo                   60000 non-null  object 
 2   name                    59998 non-null  object 
 3   blurb                   59998 non-null  object 
 4   goal                    60000 non-null  float64
 5   slug                    60000 non-null  object 
 6   disable_communication   60000 non-null  bool   
 7   country                 60000 non-null  object 
 8   currency                60000 non-null  object 
 9   currency_symbol         60000 non-null  object 
 10  currency_trailing_code  60000 non-null  bool   
 11  deadline                60000 non-null  int64  
 12  created_at              60000 non-null  int64  
 13  launched_at             60000 non-null  int64  
 14  static_usd_rate         60000 non-null

In [94]:
# I will use the following columns for my model
df = df[["goal_usd", "country", "campaign_length", "category", "state", "evaluation_set"]]

In [95]:
#Pandas Profiling to explore the selected columns
#profile = ProfileReport(df)
# profile

In [96]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
plt.switch_backend('TkAgg')

#Exploring the Category feature
cat_df = pd.get_dummies(df.set_index('category').state).groupby('category').sum() #groups the categories by either failed or succesful

#plotting the success rate by category
#fig, (ax4) = plt.subplots(3, 2, figsize=(12,12))
#cat_df.div(cat_df.sum(axis=1), axis=0).plot(kind='bar', ax=ax4) # Normalizes counts across rows
#ax4.set_title('Proportion of successful projects')
#ax4.set_xlabel('')

#fig.subplots_adjust(hspace=0.6)
#plt.show()


In [97]:
# Exploring the Country feature by one-Hot encoding - how succesful were projects per country?

country_df = pd.get_dummies(df.set_index('country').state).groupby('country').sum()

In [98]:
df = df[["goal_usd", "country", "campaign_length", "category", "state", "evaluation_set"]]

In [99]:
# Add the one_hot encoded Category features to the dataset
df = pd.get_dummies(df)
df

,goal_usd,campaign_length,state,evaluation_set,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,...,category_design,category_fashion,category_film & video,category_food,category_games,category_journalism,category_music,category_photography,category_publishing,category_technology
0,125000.0000,2489045,0.0,False,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,9800.0000,2592000,0.0,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2500.0000,1814400,1.0,False,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,16800.7930,3753610,1.0,False,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,5500.0000,2642107,0.0,False,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1800.0000,1814013,NaN,True,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,12500.0000,2592000,NaN,True,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
59997,65000.0000,2588400,NaN,True,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
59998,7127.7768,2592000,NaN,True,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [100]:
# now that all values have been formated in numerical, we define our Independant (scaled) and dependend(y) values
eval_mask = df.evaluation_set
df["evaluation_set"]
X_eval = df[eval_mask].drop(["state"], axis=1)
X_unscaled = df[~eval_mask].drop(["state"], axis=1)
y = df[~eval_mask]["state"]

In [101]:
#scale both evaluation and independant data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X_unscaled), columns=list(X_unscaled.columns))
X_eval = pd.DataFrame(scaler.fit_transform(X_eval), columns=list(X_eval.columns))

#### Train

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Prediction

In [107]:
y_pred = model.predict(X_test)
return y_pred

SyntaxError: 'return' outside function (<ipython-input-107-5a3917365509>, line 2)

##### Preprocess

In [103]:
from sklearn.linear_model import LogisticRegression

def preprocess(df):
    """This function takes a dataframe and preprocesses it so it is
    ready for the training stage.

    The DataFrame contains columns used for training (features)
    as well as the target column.

    It also contains some rows for which the target column is unknown. 
    Those are the observations you will need to predict for KATE 
    to evaluate the performance of your model.

    Here you will need to return the training set: X and y together
    with the preprocessed evaluation set: X_eval.

    Make sure you return X_eval separately! It needs to contain
    all the rows for evaluation -- they are marked with the column
    evaluation_set. You can easily select them with pandas:

         - df.loc[df.evaluation_set]

    For y you can either return a pd.DataFrame with one column or pd.Series.

    :param df: the dataset
    :type df: pd.DataFrame
    :return: X, y, X_eval
    """
    df["goal_usd"] = df["goal"] * df["static_usd_rate"]
    eval_mask = df.evaluation_set
    
    
    